In [13]:
import os
import glob

In [3]:
! ./sig2kmer.py --help

Traceback (most recent call last):
  File "./sig2kmer.py", line 20, in <module>
    from sencha.sequence_encodings import encode_peptide, AMINO_ACID_SINGLE_LETTERS
ModuleNotFoundError: No module named 'sencha'


In [4]:
! pip list | grep orpheum

In [6]:
! which -a python

/home/olga/anaconda/bin/python
/home/olga/anaconda/bin/python
/home/olga/miniconda3/envs/immune-evolution/bin/python
/usr/bin/python


In [7]:
! /home/olga/miniconda3/envs/immune-evolution/bin/python sig2kmer.py --help

usage: sig2kmer.py [-h] [--output-sequences OUTPUT_SEQUENCES]
                   [--output-kmers OUTPUT_KMERS] [--input-is-protein] [-k K]
                   [--protein] [--no-protein] [--dayhoff] [--no-dayhoff]
                   [--hp] [--no-hp] [--dna] [--no-dna]
                   query seqfiles [seqfiles ...]

positional arguments:
  query
  seqfiles

optional arguments:
  -h, --help            show this help message and exit
  --output-sequences OUTPUT_SEQUENCES
                        save matching sequences to this file.
  --output-kmers OUTPUT_KMERS
                        save matching kmers to this file.
  --input-is-protein    Consume protein sequences - no translation needed.
  -k K, --ksize K       k-mer size; default=31
  --protein             choose a protein signature; by default, a nucleotide
                        signature is used
  --no-protein          do not choose a protein signature
  --dayhoff             build Dayhoff-encoded amino acid signatures
  --no-day

In [8]:
ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures

total 11K
drwxr-xr-x  7 olga czb 4.0K Mar  5 12:02 ./
drwxrwxr-x  8 olga czb 8.0K Mar 16 10:36 ../
drwxr-xr-x  4 olga czb 4.0K Feb 28 09:53 0--mouse2mouse/
drwxr-xr-x 19 olga czb 4.0K Mar 27 13:06 1--train-mouse/
drwxr-xr-x 16 olga czb 4.0K Mar 29 06:53 2--test-human/
drwxr-xr-x 20 olga czb 4.0K Mar 29 07:04 3--test-bat/
drwxr-xr-x 18 olga czb 4.0K Apr  6 09:05 4--test-lemur/


In [29]:
ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/

total 8.0K
drwxr-xr-x 16 olga czb 4.0K Apr 16 08:44 ./
drwxr-xr-x  7 olga czb 4.0K Mar  5 12:02 ../
drwxr-xr-x  4 olga czb 4.0K Mar  3 14:58 0--fastas/
drwxr-xr-x  5 olga czb 4.0K Feb 28 11:12 1--single-cell-sigs/
drwxr-xr-x  5 olga czb 4.0K Mar 29 06:53 2--mouse-search-results/
drwxr-xr-x  5 olga czb 4.0K Mar 29 06:53 3-0--mouse-containment-search-results--min-5-percent-shared-kmers/
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:54 3-0--mouse-search-results--min-5-percent-shared-kmers/
drwxr-xr-x  5 olga czb 4.0K Mar 27 13:09 3-0--mouse-search-results--min-5-percent-shared-kmers-remove-common-kmers/
drwxr-xr-x  5 olga czb 4.0K Feb 28 17:17 3-0--mouse-search-results--remove_common/
drwxr-xr-x  5 olga czb 4.0K Mar 29 06:53 3-1--mouse-containment-search-results--min-10-percent-shared-kmers/
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:54 3-1--mouse-search-results--min-10-percent-shared-kmers/
drwxr-xr-x  5 olga czb 4.0K Mar 27 13:09 3-1--mouse-search-results--min-10-percent-shared-kmers-remove-common-k

In [12]:

ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/*/*fastas/

/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--protein-coding-fastas/:
total 2.1M
drwxr-xr-x  4 olga czb 4.0K Feb 28 10:19 ./
drwxr-xr-x 19 olga czb 4.0K Mar 27 13:06 ../
drwxr-xr-x  2 olga czb 1.0M Feb 28 10:50 nucleotides/
drwxr-xr-x  2 olga czb 1.0M Feb 28 10:19 peptides/

/home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/0--protein-coding-fastas/:
total 8.1M
drwxr-xr-x  4 olga czb 4.0K Mar  3 14:58 ./
drwxr-xr-x 16 olga czb 4.0K Mar 29 06:53 ../
drwxr-xr-x  2 olga czb 4.0M Mar  5 16:42 nucleotides/
drwxr-xr-x  2 olga czb 4.0M Mar  5 16:38 peptides/

/home/olga/data_sm/immune-evolution/kmer-signatures/3--test-bat/0--protein-coding-fastas/:
total 4.3M
drwxr-xr-x  4 olga czb 4.0K Mar  5 16:29 ./
drwxr-xr-x 20 olga czb 4.0K Mar 29 07:04 ../
drwxr-xr-x  2 olga czb 2.3M Mar  5 17:27 nucleotides/
drwxr-xr-x  2 olga czb 2.0M Mar  5 16:57 peptides/

/home/olga/data_sm/immune-evolution/kmer-signatures/4--test-lemur/0--protein-coding-fastas/:
total 2.1M


In [14]:
glob.glob('/home/olga/data_sm/immune-evolution/kmer-signatures/*/*fastas/')

['/home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/0--protein-coding-fastas/',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/3--test-bat/0--protein-coding-fastas/',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/4--test-lemur/0--protein-coding-fastas/',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--protein-coding-fastas/']

In [15]:
for fasta_dir_original in glob.glob('/home/olga/data_sm/immune-evolution/kmer-signatures/*/*fastas/'):
    fasta_dir_new = fasta_dir_original.replace('protein-coding-', '')
    ! mv $fasta_dir_original $fasta_dir_new

In [18]:
glob.glob('/home/olga/data_sm/immune-evolution/kmer-signatures/*/1--single-cell-sigs/')

['/home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/1--single-cell-sigs/',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/3--test-bat/1--single-cell-sigs/',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/4--test-lemur/1--single-cell-sigs/',
 '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/1--single-cell-sigs/']

In [25]:
! ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/1--single-cell-sigs/ | head

total 1.6M
drwxr-xr-x  5 olga czb 4.0K Feb 28 09:34 .
drwxr-xr-x 19 olga czb 4.0K Apr 16 08:44 ..
drwxr-xr-x  2 olga czb 512K Feb 28 09:32 alphabet-DNA__ksize-21__scaled-10
drwxr-xr-x  2 olga czb 512K Feb 28 09:35 alphabet-dayhoff__ksize-51__scaled-10
drwxr-xr-x  2 olga czb 512K Feb 28 09:34 alphabet-protein__ksize-30__scaled-10


In [27]:
! ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/1--single-cell-sigs/alphabet-dayhoff__ksize-51__scaled-10 | head

total 838M
drwxr-xr-x 2 olga czb 512K Feb 28 09:35 .
drwxr-xr-x 5 olga czb 4.0K Feb 28 09:34 ..
-rw-rw-r-- 1 olga czb 413K Feb 28 09:34 MACA_18m_F_LUNG_50__AAACGGGTCGCCCTTA.sig
-rw-rw-r-- 1 olga czb 583K Feb 28 09:34 MACA_18m_F_LUNG_50__AAAGCAACATGGTAGG.sig
-rw-rw-r-- 1 olga czb 201K Feb 28 09:34 MACA_18m_F_LUNG_50__AAAGCAAGTCTAGTGT.sig
-rw-rw-r-- 1 olga czb 194K Feb 28 09:34 MACA_18m_F_LUNG_50__AAATGCCAGGAGTTGC.sig
-rw-rw-r-- 1 olga czb 858K Feb 28 09:34 MACA_18m_F_LUNG_50__AAATGCCGTCTGCCAG.sig
-rw-rw-r-- 1 olga czb 210K Feb 28 09:34 MACA_18m_F_LUNG_50__AAATGCCTCAGCCTAA.sig
-rw-rw-r-- 1 olga czb 658K Feb 28 09:34 MACA_18m_F_LUNG_50__AACACGTCATTAGGCT.sig
ls: write error: Broken pipe


In [24]:
! ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--fastas/peptides | head

total 68G
drwxr-xr-x 2 olga czb 1.0M Feb 28 10:19 .
drwxr-xr-x 4 olga czb 4.0K Feb 28 10:19 ..
-rw-rw-r-- 1 olga czb  80M Feb 28 10:06 MACA_18m_F_LUNG_50__aligned__aligned__AAACGGGTCGCCCTTA__coding_reads_peptides.fasta
-rw-rw-r-- 1 olga czb 108M Feb 28 09:51 MACA_18m_F_LUNG_50__aligned__aligned__AAAGCAACATGGTAGG__coding_reads_peptides.fasta
-rw-rw-r-- 1 olga czb  32M Feb 28 10:19 MACA_18m_F_LUNG_50__aligned__aligned__AAAGCAAGTCTAGTGT__coding_reads_peptides.fasta
-rw-rw-r-- 1 olga czb  34M Feb 28 09:59 MACA_18m_F_LUNG_50__aligned__aligned__AAATGCCAGGAGTTGC__coding_reads_peptides.fasta
-rw-rw-r-- 1 olga czb 177M Feb 28 09:49 MACA_18m_F_LUNG_50__aligned__aligned__AAATGCCGTCTGCCAG__coding_reads_peptides.fasta
-rw-rw-r-- 1 olga czb  36M Feb 28 10:07 MACA_18m_F_LUNG_50__aligned__aligned__AAATGCCTCAGCCTAA__coding_reads_peptides.fasta
-rw-rw-r-- 1 olga czb 149M Feb 28 10:07 MACA_18m_F_LUNG_50__aligned__aligned__AACACGTCATTAGGCT__coding_reads_peptides.fasta
ls: write error: Broken pipe


In [26]:
! ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--fastas | head

total 2.1M
drwxr-xr-x  4 olga czb 4.0K Feb 28 10:19 .
drwxr-xr-x 19 olga czb 4.0K Apr 16 08:44 ..
drwxr-xr-x  2 olga czb 1.0M Feb 28 10:50 nucleotides
drwxr-xr-x  2 olga czb 1.0M Feb 28 10:19 peptides


# Single-cell signatures

In [20]:
pwd

'/home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks'

In [60]:
PYTHON = '/home/olga/miniconda3/envs/immune-evolution/bin/python'
PWD = '/home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks'
SIG2KMER = f"{PWD}/sig2kmer.py"
sig2kmer_template = f"{PYTHON} {SIG2KMER} " + r"--quiet {moltype_flags} --output-sequences {output_fasta} --output-kmers {output_kmer_csv} {query_sig} {input_fasta}"
sig2kmer_template

'/home/olga/miniconda3/envs/immune-evolution/bin/python /home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks/sig2kmer.py --quiet {moltype_flags} --output-sequences {output_fasta} --output-kmers {output_kmer_csv} {query_sig} {input_fasta}'

In [61]:
fasta_type_to_sketch_ids = {
    "nucleotides": ["alphabet-DNA__ksize-21__scaled-10"],
    "peptides": [
        "alphabet-dayhoff__ksize-51__scaled-10",
        "alphabet-protein__ksize-30__scaled-10",
    ],
}

sketch_id_to_flags = {
    "alphabet-DNA__ksize-21__scaled-10": "--dna --ksize 21",
    "alphabet-dayhoff__ksize-51__scaled-10": "--dayhoff --no-dna --ksize 51 --input-is-protein",
    "alphabet-protein__ksize-30__scaled-10": "--protein --no-dna --ksize 30 --input-is-protein",
}

In [ ]:


# Add "--t" to glob command for train and test data
for species_folder in glob.glob(
    "/home/olga/data_sm/immune-evolution/kmer-signatures/*--test-bat*/"
):
    fasta_base = os.path.join(species_folder, "0--fastas")
    sig_base = os.path.join(species_folder, "1--single-cell-sigs")
    output_base = os.path.join(species_folder, "2--single-cell-kmers")
    if not os.path.exists(output_base):
        os.makedirs(output_base)
    
    
    command_txt = os.path.join(output_base, 'sig2kmer_commands.txt')
    
    with open(command_txt, 'w') as f:
        for fasta_type, sketch_ids in fasta_type_to_sketch_ids.items():
            input_fastas = glob.glob(os.path.join(fasta_base, fasta_type, '*'))
            for sketch_id in sketch_ids:
                moltype_flags = sketch_id_to_flags[sketch_id]
                sketch_id_sig_folder = os.path.join(sig_base, sketch_id)

                sketch_id_output_base = os.path.join(output_base, sketch_id)
                fasta_output_base = os.path.join(sketch_id_output_base, "fastas")
                csv_output_base = os.path.join(sketch_id_output_base, "csvs")
                
#                 if not os.path.exists(csv_output_base):
#                     os.makedirs(fasta_output_base)
#                     os.makedirs(csv_output_base)

                for input_fasta in input_fastas:
                    basename = os.path.basename(input_fasta)
                    alignment_status = 'aligned' if '_aligned_' in basename else 'unaligned'
                    fasta_alignment_status_base = os.path.join(fasta_output_base, alignment_status)
                    csv_alignment_status_base = os.path.join(csv_output_base, alignment_status)
                    
                    if not os.path.exists(csv_alignment_status_base):
                        os.makedirs(fasta_alignment_status_base)
                        os.makedirs(csv_alignment_status_base)
                    
                    cell_id = basename.replace("__aligned", "").replace('__unaligned', '').split("__coding_reads")[0]

                    # Create inputs
                    query_sig = os.path.join(sketch_id_sig_folder, f"{cell_id}.sig")
                    
                    print(cell_id)
                    break
                    
                    if not os.path.exists(query_sig):
                        # Only used signatures from pre-selected celltypes --> ignore everything else
                        ! rm $input_fasta
                        continue
                    output_fasta = os.path.join(fasta_alignment_status_base, f"{cell_id}.fasta")
                    output_kmer_csv = os.path.join(csv_alignment_status_base, f"{cell_id}.csv")

                    command = sig2kmer_template.format(
                        moltype_flags=moltype_flags,
                        output_fasta=output_fasta,
                        output_kmer_csv=output_kmer_csv,
                        query_sig=query_sig,
                        input_fasta=input_fasta,
                    )
                    f.write(command + '\n')

    print(f'parallel --progress --eta --jobs 96 < {command_txt}')
    ! wc -l $command_txt
    print('---')
    ! head -n 1 $command_txt
    print('---')
                


In [57]:
cell_id

'MACA_21m_F_LUNG_55__GGCAATTAGGCATGTG'

In [53]:
input_fasta

'/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--fastas/peptides'

In [48]:
input_fastas

['/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--fastas/peptides']

In [41]:
fasta_type

'peptides'

# Per-celltype database signatures for diagnostic k-mers